# Azure ML - Jupyter Notebooks

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID, workspace_name)

## Scripts

Una vez que hemos validado el código de entrenamiento en un Jupyter Notebook podemos pasarlo a un script para ejecutarlo y poder hacer varios experimentos cambiando hiperparámetros

### Ejecutar un script desde la interfaz gráfica

#### Subir los datos

Nos vamos a `Notebooks` en la zona de las carpetas hay un botón con el símbolo `+`, si le damos podemos subir archivos. Vamos a subir la carpeta `en` que descargamos de HuggingFace, marcamos la casilla que dice si creemos en los autores y le damos a `Upload`. Ya tenemos los datos subidos

#### Crear el script

Volvemos a dar al botón con el símbolo `+`, le damos a `Create new file`, en `File type` seleccionamos `Python (*.py)` y le ponemos el nombre `text-classification.py`. Ahora copiamos el siguiente código

```python
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import Accuracy
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage
import argparse
from datetime import datetime

def load_dataset():
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor()
    )

    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor()
    )
    return training_data, test_data

def get_num_classes(train_dataset):
    set_clases = set({})
    len_training_data = len(train_dataset)
    for i in range(len_training_data):
        _, label = train_dataset[i]
        set_clases.add(label)
    num_classes = len(set_clases)
    return num_classes

def create_subset(tran_dataset, test_dataset, factor):
    len_training_data = len(tran_dataset)
    len_test_data = len(test_dataset)
    subset_training_data = torch.utils.data.Subset(tran_dataset, range(0, len_training_data//factor))
    subset_test_data = torch.utils.data.Subset(test_dataset, range(0, len_test_data//factor))
    return subset_training_data, subset_test_data

def get_dataloaders(train_dataset, test_dataset, batch_size):
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    return train_dataloader, test_dataloader

class ImageClassifier(nn.Module):
    def __init__(self, num_classes):
        self.num_classes = num_classes
        super(ImageClassifier, self).__init__()
        self.imageClassifier = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.LazyLinear(self.num_classes),
        )

    def forward(self, x):
        return self.imageClassifier(x)

def train(dataloader, model, loss_fn, metrics_fn, optimizer, epoch, device):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)
        metric = metrics_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss = loss.item()
            current = batch
            step = batch // 1000 * (epoch + 1)
            print(f"train loss: {loss:.4f}, train accuracy: {metric:.4f} [{current}/{len(dataloader)}]")

    return model

def evaluate(dataloader, model, loss_fn, metrics_fn, epoch, device):
    num_batches = len(dataloader)
    model.eval()
    eval_loss = 0
    eval_accuracy = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            eval_loss += loss_fn(pred, y).item()
            eval_accuracy += metrics_fn(pred, y).item()

    eval_loss /= num_batches
    eval_accuracy /= num_batches
    print(f"eval loss: {eval_loss:.4f}, eval accuracy: {eval_accuracy:.4f}")

def main(subset_factor, batch_size, epochs=3, learning_rate=1e-3):
    train_dataset, test_dataset = load_dataset()
    num_classes = get_num_classes(train_dataset)
    train_subset, test_subset = create_subset(train_dataset, test_dataset, factor=subset_factor)

    train_dataloader, test_dataloader = get_dataloaders(train_subset, test_subset, batch_size=batch_size)
    sample_batch = next(iter(train_dataloader))
    image_batch, _ = sample_batch

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = ImageClassifier(num_classes).to(device)

    loss_fn = nn.CrossEntropyLoss()
    metrics_fn = Accuracy(task="multiclass", num_classes=num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, metrics_fn, optimizer, epoch, device)
        evaluate(test_dataloader, model, loss_fn, metrics_fn, epoch, device)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--subset_factor", type=int, default=1)
    parser.add_argument("--batch_size", type=int, default=16)
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--learning_rate", type=float, default=1e-3)
    args = parser.parse_args()

    subset_factor = args.subset_factor
    batch_size = args.batch_size
    epochs = args.epochs
    learning_rate = args.learning_rate

    main(subset_factor, batch_size, epochs, learning_rate)
```

#### Ejecutar el script

Volvemos a dar al botón con el símbolo `+`, le damos a `Create new file`, en `File type` seleccionamos `Notebook (*.ipynb)` y le ponemos el nombre `run_text-clasification.ipynb`. Ahora copiamos las siguientes celdas de código

```python
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except:
    credential = InteractiveBrowserCredential()
```

```python
ml_client = MLClient.from_config(credential=credential)
```

```python
from azure.ai.ml import command

execute_command = "python image-classification.py --batch_size 16 --epochs 3 --learning_rate 1e-3"
environment = "cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_GUI@latest"
compute_instance = "compute-instance-GUI"
display_name = "image-classificator"
experiment_name = "image-classification"

# configure job
job = command(
    code="./",
    command=execute_command,
    environment=environment,
    compute=compute_instance,
    display_name=display_name,
    experiment_name=experiment_name
)
```

```python
returned_job = ml_client.create_or_update(job)
```

Mira que en el environment hemos puesto `@latest` para que use la última versión del entorno

Otra cosa importate es que los hiperparámetros como el batch size, el learning rate, etc. se pasan al script como argumentos, por lo que puedes hacer muchos experimentos modificando estos valores en los argumentos del script

Seleccionamos la compute instance `compute-instance-GUI` y ejecutamos todas las celdas, si todo ha ido bien, se habrá creado un nuevo experimento en Azure ML

#### Visualizar el entrenamiento

Si nos vamos a la sección `Jobs` en la parte izquierda de la pantalla, veremos el experimento que hemos creado, si le damos veremos la salida del script.

### Ejecutar un script con el CLI de Azure ML

No he encotrado la manera de ejecutar el script mediante la CLI de Azure ML

### Ejecutar un script con el SDK de Python

Para ejecutar un script primero tenemos que subir los datos y el script igual que lo hemos hecho en la interfaz gráfica. No he encotrado la manera de subirlos mediante el SDK de Python. Por lo que nos aseguramos de estar en el workspace en el que hemos hecho todo con el SDK de Python y subimos los datos y el script igual que lo hemos hecho antes

Ahora para ejecutar ek script ejecutamos ek mismo código que ejecutamos en el notebook mediante la interfaz gráfica

In [ ]:
from azure.ai.ml import command

execute_command = "python image-classification.py --batch_size 16 --epochs 3 --learning_rate 1e-3"
environment = "cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_G@latest"
compute_instance = "compute-instance-Python"
display_name = "text-clasificator"
experiment_name = "train-classification-model"

# configure job
job = command(
    code="./",
    command=execute_command,
    environment=environment,
    compute=compute_instance,
    display_name=display_name,
    experiment_name=experiment_name
)

In [5]:
returned_job = ml_client.create_or_update(job)